## Introduction to Voyage & TimeSeries endpoint

The aim of this notebook is to illustrate how Vortexa clients could use the SDK. For the purposes of this analysis we will take advantage of the SDK's [VoyageTimeSeries](https://vortechsa.github.io/python-sdk/endpoints/voyages_timeseries/), [VoyageSearchEnriched](https://vortechsa.github.io/python-sdk/endpoints/voyages_search_enriched/) endpoint.

## Import Libraries

In [1]:
import vortexasdk as v
import pandas as pd
from datetime import datetime
import plotly.express as px
import numpy as np

## Extract relevant id

In [2]:
# Extracting malaysia id
australia = v.Geographies().search('Australia',True).to_df().loc[0,'id']
egypt = v.Geographies().search('Egypt',True).to_df().loc[0,'id']
china = v.Geographies().search('China',True).to_df().loc[0,'id']

# Extracting product id
cpp = v.Products().search('Clean Petroleum Products',exact_term_match = True).to_df().loc[:,'id'].to_list()
dpp = v.Products().search('Dirty Petroleum Products',exact_term_match = True).to_df().loc[:,'id'].to_list()
lpg = v.Products().search('LPG+',exact_term_match = True).to_df().loc[:,'id'].to_list()
propane = v.Products().search('Propane',exact_term_match = True).to_df().loc[:,'id'].to_list()
butane = v.Products().search('Butane',exact_term_match = True).to_df().loc[:,'id'].to_list()
urals = v.Products().search('Urals',exact_term_match = True).to_df().loc[:,'id'].to_list()
assert len(cpp) == 1
assert len(dpp) == 1

## Helper function

In [3]:
def transform_df(df,label):
    df = df.loc[:,~df.columns.str.endswith('id')]
    df2 = df.drop(['count','value'],axis = 1)
    df2 = df2.set_index('key')
    df2.columns = pd.MultiIndex.from_tuples([(col.split('.')[2],col.split('.')[1]) for col in df2.columns])
    df3 = df2.stack().reset_index()
    df3 = df3.rename(columns = {'key':'date','label' : label})
    df3.drop(['level_1'],axis = 1,inplace = True)
    return df3

## Preview of our voyage data

## 1. Dataframe format

In [4]:
voyage_df = v.VoyagesSearchEnriched().search(
    latest_products = cpp,
    time_min = datetime(2022,1,1),
    vessels = 'oil_aframax_lr2',
    time_max = datetime(2022,12,31,23,59,59),
    origins = china,
    columns = 'all'
    ).to_df()

In [5]:
voyage_df

,VESSEL NAME,IMO,DWT (t),CAPACITY (cbm),VESSEL CLASS,VOYAGE STATUS,ORIGIN,ORIGIN TERMINAL,ORIGIN PORT,ORIGIN COUNTRY,...,FINAL DESTINATION,FINAL DESTINATION TERMINAL,FINAL DESTINATION PORT,FINAL DESTINATION COUNTRY,FINAL DESTINATION BASIN,FINAL DESTINATION SHIPPING REGION,FINAL DESTINATION WIDER SHIPPING REGION,FINAL DESTINATION COUNTRY ZONE,FINAL DESTINATION ALTERNATIVE REGION,FINAL DESTINATION STATE OR PROVINCE
0,ZALIV AMURSKIY,9354313,106855,124112,LR2,Ballast,Dalian [CN],Fujia Dahua Petrochemical Terminal,Dalian [CN],China,...,Nakhodka [RU],Rosneft-Nakhodkanefteproduct,Nakhodka [RU],Russia,Pacific Basin,Russia Far East,Wider Northeast Asia (WNEA),Russia Far East,Far East,
1,YUAN LIAN WAN,9845960,113800,126280,LR2,Laden,Yangpu [CN],Vopak SDIC Yangpu Terminal,Yangpu [CN],China,...,Singapore [SG],ExxonMobil Singapore Refinery (PAC),Singapore [SG],Singapore,Pacific Basin,Southeast Asia (SEA),Wider Southeast Asia (WSEA),,Far East,
2,YUAN LAN WAN,9845946,109900,125859,LR2,Laden,"Qingdao [CN], Tianjin [CN], Pengerang [MY]","Huangdao Oil Terminal, Tianjin Petrochemicals ...","Qingdao [CN], Tianjin [CN], Pengerang [MY]","China, China, Malaysia",...,Antwerp [BE],Sea-Tank Terminal Antwerp (Quay 510),Antwerp [BE],Belgium,Atlantic Basin,Northwest Europe (NWE),Wider Northwest Europe (WNWE),,North Sea,
3,YUAN LAN WAN,9845946,109900,125859,LR2,Laden,Dalian [CN],PetroChina Dalian Petrochemical,Dalian [CN],China,...,Singapore [SG],Jurong Port Tank Terminal,Singapore [SG],Singapore,Pacific Basin,Southeast Asia (SEA),Wider Southeast Asia (WSEA),,Far East,
4,YUAN JU WAN,9845958,109900,125518,LR2,Laden,"Tianjin [CN], Qingdao [CN]",", Huangdao Oil Terminal","Tianjin [CN], Qingdao [CN]","China, China",...,Pengerang [MY],Pengerang Deepwater Petroleum Terminal,Pengerang [MY],Malaysia,Pacific Basin,Southeast Asia (SEA),Wider Southeast Asia (WSEA),,Far East,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,AL DASMA,9653446,109719,124683,LR2,Ballast,Huizhou [CN],CNOOC - Shell Mabianzhou Crude Petrochemical T...,Huizhou [CN],China,...,Mai Liao - Taiwan [TW],Mailiao Industrial Park-Formosa Plastics,Mai Liao - Taiwan [TW],Taiwan,Pacific Basin,Northeast Asia (NEA),Wider Northeast Asia (WNEA),,Far East,
167,AL DASMA,9653446,109719,124683,LR2,Laden,Dalian [CN],PetroChina Dalian Petrochemical,Dalian [CN],China,...,Singapore [SG],Jurong Port Universal Terminal (JPUT),Singapore [SG],Singapore,Pacific Basin,Southeast Asia (SEA),Wider Southeast Asia (WSEA),,Far East,
168,ADVANTAGE LIFE,9708564,109999,125294,LR2,Laden,"Qingdao [CN], Pengerang [MY]","Huangdao Oil Terminal, Pengerang Deepwater Pet...","Qingdao [CN], Pengerang [MY]","China, Malaysia",...,Antwerp [BE],Oiltanking Stolthaven Antwerp,Antwerp [BE],Belgium,Atlantic Basin,Northwest Europe (NWE),Wider Northwest Europe (WNWE),,North Sea,
169,A LEOPARD,9838589,114000,124696,LR2,Laden,Yangpu [CN],Hainan SINOPEC Oil Refinery Terminal,Yangpu [CN],China,...,Mai Liao - Taiwan [TW],Mailiao Industrial Park-Formosa Plastics,Mai Liao - Taiwan [TW],Taiwan,Pacific Basin,Northeast Asia (NEA),Wider Northeast Asia (WNEA),,Far East,


In [6]:
voyage_df.columns

Index(['VESSEL NAME', 'IMO', 'DWT (t)', 'CAPACITY (cbm)', 'VESSEL CLASS',
       'VOYAGE STATUS', 'ORIGIN', 'ORIGIN TERMINAL', 'ORIGIN PORT',
       'ORIGIN COUNTRY', 'ORIGIN BASIN', 'ORIGIN SHIPPING REGION',
       'ORIGIN WIDER SHIPPING REGION', 'ORIGIN COUNTRY ZONE',
       'ORIGIN ALTERNATIVE REGION', 'ORIGIN STATE OR PROVINCE', 'DESTINATION',
       'DESTINATION TERMINAL', 'DESTINATION PORT', 'DESTINATION COUNTRY',
       'DESTINATION BASIN', 'DESTINATION SHIPPING REGION',
       'DESTINATION WIDER SHIPPING REGION', 'DESTINATION COUNTRY ZONE',
       'DESTINATION ALTERNATIVE REGION', 'DESTINATION STATE OR PROVINCE',
       'START DATE', 'END DATE', 'LATEST PRODUCT', 'LATEST PRODUCT GROUP',
       'LATEST PRODUCT CATEGORY', 'LATEST PRODUCT GRADE', 'QUANTITY (bbl)',
       'CHARTERER', 'EFFECTIVE CONTROLLER', 'TIME CHARTERER', 'BUILD YEAR',
       'FLAG', 'RISK RATING', 'SCRUBBERS', 'COATING', 'TONNE-MILES',
       'DURATION (h)', 'DISTANCE', 'VOYAGE ID', 'PREVIOUS VOYAGE ID',
     

## 2. Json format (object)

In [7]:
voyage_df = v.VoyagesSearchEnriched().search(
    latest_products = cpp,
    time_min = datetime(2022,1,1),
    vessels = 'oil_aframax_lr2',
    time_max = datetime(2022,1,31,23,59,59),
    ).to_list()

In [8]:
voyage_df[0]

VoyageEnrichedItem(voyage_id='56be40a133a5ea2b2e4e4923bed85e7542f94b78734aa899bd30e10d318a394c', vessel_id='b802c14e852bb8ecae86f43950ced0f7272d4e5bef252adaf119f68af5980e83', schema_version='1.0.0', start_event_id='f0022f0e62eff6a260d7d609adf3fe31f36e6b3a068a06f227407ee1c09e373d', vessel=VoyagesVesselEntity(id='b802c14e852bb8ecae86f43950ced0f7272d4e5bef252adaf119f68af5980e83', name='ZHEN HUA 35', dead_weight=105846, vessel_class='oil_aframax', classes=[VesselClassEntry(id='oil', layer='group', label='Oil Tankers'), VesselClassEntry(id='oil_aframax_lr2', layer='coarse', label='Aframax / LR2'), VesselClassEntry(id='oil_aframax', layer='granular', label='Aframax')], imo=9237802, mmsi=636018736, call_sign='D5RD7', cubic_capacity=118034, year=2003, flag=[Flag(tag='vessel_flag_tag', flag='LR', flag_country='76044b057b5a5157476aff1aa4d2a6a7f5d0464cb5f5e6264ed6e3f9f27b01e3')], scrubber=[], ice_class=None, propulsion=None, tags=[], vessel_risk_level='low'), start_timestamp='2021-05-11T06:36:12.

In [9]:
pd.DataFrame(voyage_df[0])

,0,1
0,voyage_id,56be40a133a5ea2b2e4e4923bed85e7542f94b78734aa8...
1,vessel_id,b802c14e852bb8ecae86f43950ced0f7272d4e5bef252a...
2,schema_version,1.0.0
3,start_event_id,f0022f0e62eff6a260d7d609adf3fe31f36e6b3a068a06...
4,vessel,id='b802c14e852bb8ecae86f43950ced0f7272d4e5bef...
5,start_timestamp,2021-05-11T06:36:12.000Z
6,end_timestamp,2022-04-12T04:08:35.000Z
7,end_event_id,c4c216322948633c276da0cb101fdb358c567d1d3baa0d...
8,previous_voyage_id,f81b8579b2065017b424c1f22f04a63ff81b8579b20650...
9,next_voyage_id,b423e5bb36d40654dec884aa7a7153fdb423e5bb36d406...


In [10]:
events_df = pd.DataFrame([t.__dict__ for t in voyage_df[0].events])
events_df

,event_id,value,event_group,event_type,location_id,start_timestamp,end_timestamp,activity,odometer_start,odometer_end,location_layer,cargo_movement_id,sts_event_counterparty_vessel_id,waiting_event_target_geography_id,fixture_event_fixing_timestamp,tags,probability,location_details,is_open_event,waiting_event_target_geography_details
0,3a6a2fa9ea33742d654a57a2d1546f3f3a675d8820fb90...,None,vessel,stationary_event,0011001100110011,2021-05-11T06:36:12.000Z,2021-05-11T10:56:00.000Z,None,9.391925e+08,9.394119e+08,[root],[],None,None,None,[],1.0,[],None,[]
1,f761e405da440c3394c7b3ef02c99e78e0cbe9f6a9f636...,None,vessel,visit,7fed43c640957555,2021-05-11T06:36:12.000Z,2021-05-11T07:25:52.000Z,None,9.392478e+08,9.393576e+08,[country],[],None,None,None,[],1.0,[id='7fed43c640957555' layer='country' label='...,None,[]
2,f0022f0e62eff6a260d7d609adf3fe31f36e6b3a068a06...,None,vessel,point_in_time,1b79e18416d358d7,2021-05-11T06:36:12.000Z,2021-05-11T06:36:12.000Z,None,9.393480e+08,9.393480e+08,"[port, storage]",[],None,None,None,[tag='voyage_start' start_timestamp=None end_t...,1.0,[id='1b79e18416d358d7' layer='port' label='Sin...,None,[]
3,c780749c192bf793,on-the-sea,derived,status,None,2021-05-11T06:36:12.000Z,2021-05-11T07:25:52.000Z,location,NaN,NaN,None,None,None,None,None,None,NaN,None,False,None
4,0011001100110011,unknown,derived,status,None,2021-05-11T06:36:12.000Z,2022-04-12T04:08:35.000Z,commitment,NaN,NaN,None,None,None,None,None,None,NaN,None,False,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,d387ce2af90c1ea3,moving,derived,status,None,2022-04-10T22:38:30.000Z,2022-04-12T00:46:00.000Z,movement,NaN,NaN,None,None,None,None,None,None,NaN,None,False,None
553,b17690e7fef466ebe43939cbcf2ac3a558ed9ea89b7ba6...,None,vessel,visit,0a9f381e26778f77,2022-04-12T04:08:35.000Z,2022-04-12T04:08:35.000Z,None,1.027233e+09,1.027558e+09,[fragment],[],None,None,None,[],1.0,[id='0a9f381e26778f77' layer='fragment' label=...,None,[]
554,c4c216322948633c276da0cb101fdb358c567d1d3baa0d...,None,vessel,point_in_time,02c394639ce941b9,2022-04-12T04:08:35.000Z,2022-04-12T04:08:35.000Z,None,1.027233e+09,1.027233e+09,"[port, storage]",[],None,None,None,[tag='voyage_end' start_timestamp=None end_tim...,1.0,[id='02c394639ce941b9' layer='port' label='Ash...,None,[]
555,96229ede124e4bbf,on-the-sea,derived,status,None,2022-04-12T04:08:35.000Z,2022-04-12T04:08:35.000Z,location,NaN,NaN,None,None,None,None,None,None,NaN,None,False,None


In [11]:
events_df[events_df['event_type']=='visit']

,event_id,value,event_group,event_type,location_id,start_timestamp,end_timestamp,activity,odometer_start,odometer_end,location_layer,cargo_movement_id,sts_event_counterparty_vessel_id,waiting_event_target_geography_id,fixture_event_fixing_timestamp,tags,probability,location_details,is_open_event,waiting_event_target_geography_details
1,f761e405da440c3394c7b3ef02c99e78e0cbe9f6a9f636...,None,vessel,visit,7fed43c640957555,2021-05-11T06:36:12.000Z,2021-05-11T07:25:52.000Z,None,9.392478e+08,9.393576e+08,[country],[],None,None,None,[],1.0,[id='7fed43c640957555' layer='country' label='...,None,[]
7,8b7616cbea5e3a5e44636241424b356ffb62f9972c7328...,None,vessel,visit,612eeab9024bfb73,2021-05-11T07:26:22.000Z,2021-05-11T12:48:01.120Z,None,9.393576e+08,9.394524e+08,[country],[],None,None,None,[],1.0,[id='612eeab9024bfb73' layer='country' label='...,None,[]
9,6311b2c55dd5cfef3143ef1c8d16c2502e9c3ea595f2e8...,None,vessel,visit,e5b18283df1fad31,2021-05-11T07:33:04.000Z,2021-05-11T08:36:32.030Z,None,9.393576e+08,9.393732e+08,[sts_zone],[],None,None,None,[],1.0,[id='e5b18283df1fad31' layer='sts_zone' label=...,None,[]
10,0b7a624bc33ca7fef446997916ce356a9f7cdbb41f33cf...,None,vessel,visit,3267ef2a83a74905,2021-05-11T12:52:30.000Z,2021-05-11T19:17:51.870Z,None,9.394524e+08,9.396080e+08,[country],[],None,None,None,[],1.0,[id='3267ef2a83a74905' layer='country' label='...,None,[]
12,7f2b17c87806711b06c46152209cc7899cb3264d16af85...,None,vessel,visit,afaa601a73ea4d26,2021-05-11T13:42:51.000Z,2021-05-11T18:17:52.010Z,None,9.394743e+08,9.395838e+08,[sts_zone],[],None,None,None,[],1.0,[id='afaa601a73ea4d26' layer='sts_zone' label=...,None,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541,318d06a00eeabaa213232f046bb6fa85881ad1f7eac2be...,None,vessel,visit,b6be463f6999751d,2022-04-09T12:45:50.000Z,2022-04-09T17:39:38.000Z,None,1.025772e+09,1.025889e+09,[country],[],None,None,None,[],1.0,[id='b6be463f6999751d' layer='country' label='...,None,[]
544,2f4b0e18c00c2c039072720df3d8908970eaa791dae5f1...,None,vessel,visit,82a6dd5bf8dfb66a,2022-04-09T17:41:19.000Z,2022-04-09T22:38:53.000Z,None,1.025889e+09,1.026002e+09,[country],[],None,None,None,[],1.0,[id='82a6dd5bf8dfb66a' layer='country' label='...,None,[]
547,d4c13034b592a6d258a0dbf9eec1204ba6ebaf99be6e4d...,None,vessel,visit,b6be463f6999751d,2022-04-09T22:39:33.000Z,2022-04-10T22:32:40.000Z,None,1.026002e+09,1.026555e+09,[country],[],None,None,None,[],1.0,[id='b6be463f6999751d' layer='country' label='...,None,[]
550,b3526aae64c9cdc7cf4b7cbfc84a431eb295105fb4dcf8...,None,vessel,visit,ededa118cc31a7f7,2022-04-10T22:38:30.000Z,2022-04-12T00:46:00.000Z,None,1.026555e+09,1.027157e+09,[],[],None,None,None,[],1.0,[id='3eac69e760d9ec57' layer='country' label='...,None,[]


## 1. CPP sailing aframax split by vessel status (laden/ballast)

In [12]:
def extract_voyage():    
    df1 = v.VoyagesTimeseries().search(
        breakdown_frequency = 'day', ## week, month, year
        latest_products = cpp,
        breakdown_property = 'vessel_count', ## tonne-miles or etc
        breakdown_split_property = 'vessel_status', # ballast or laden
        time_min = datetime(2023,1,1),
        vessels = 'oil_aframax_lr2',
        time_max = datetime.today(),
        movement_status = 'moving', ## congestion, waiting, stationary
        ).to_df(columns = 'all')
    return df1

In [13]:
df1 = extract_voyage()

In [14]:
df1

,key,value,count,breakdown.0.id,breakdown.0.label,breakdown.0.count,breakdown.0.value,breakdown.1.id,breakdown.1.label,breakdown.1.count,breakdown.1.value
0,2023-01-01 00:00:00+00:00,176,176,ballast,ballast,59,59,laden,laden,117,117
1,2023-01-02 00:00:00+00:00,174,174,ballast,ballast,54,54,laden,laden,120,120
2,2023-01-03 00:00:00+00:00,176,176,ballast,ballast,51,51,laden,laden,125,125
3,2023-01-04 00:00:00+00:00,181,181,ballast,ballast,50,50,laden,laden,131,131
4,2023-01-05 00:00:00+00:00,184,184,ballast,ballast,57,57,laden,laden,127,127
...,...,...,...,...,...,...,...,...,...,...,...
536,2024-06-20 00:00:00+00:00,200,201,ballast,ballast,79,79,laden,laden,122,122
537,2024-06-21 00:00:00+00:00,203,204,ballast,ballast,78,78,laden,laden,126,126
538,2024-06-22 00:00:00+00:00,195,195,ballast,ballast,77,77,laden,laden,118,118
539,2024-06-23 00:00:00+00:00,216,216,ballast,ballast,83,83,laden,laden,133,133


In [15]:
transform_df1= transform_df(df1,'vessel_status')

In [16]:
transform_df1

,date,count,vessel_status,value
0,2023-01-01 00:00:00+00:00,59,ballast,59
1,2023-01-01 00:00:00+00:00,117,laden,117
2,2023-01-02 00:00:00+00:00,54,ballast,54
3,2023-01-02 00:00:00+00:00,120,laden,120
4,2023-01-03 00:00:00+00:00,51,ballast,51
...,...,...,...,...
1077,2024-06-22 00:00:00+00:00,118,laden,118
1078,2024-06-23 00:00:00+00:00,83,ballast,83
1079,2024-06-23 00:00:00+00:00,133,laden,133
1080,2024-06-24 00:00:00+00:00,100,ballast,100


In [17]:
px.bar(transform_df1,x = 'date', y = 'count',color = 'vessel_status',title = 'CPP sailing Aframax split by vessel status')

## 2. Laden sailing aframax split by origin

In [18]:
df2 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    breakdown_split_property = 'origin_region',
    time_min = datetime(2022,1,1),
    time_max = datetime(2022,12,31,23,59,59),
    movement_status = 'moving',
    vessels = 'oil_aframax_lr2',
    voyage_status = 'laden'
    ).to_df(columns = 'all')

In [19]:
df2

,key,value,count,breakdown.0.id,breakdown.0.label,breakdown.0.count,breakdown.0.value,breakdown.1.id,breakdown.1.label,breakdown.1.count,...,breakdown.7.count,breakdown.7.value,breakdown.8.id,breakdown.8.label,breakdown.8.count,breakdown.8.value,breakdown.9.id,breakdown.9.label,breakdown.9.count,breakdown.9.value
0,2022-01-01 00:00:00+00:00,346,346,c4a371acd7c3df8b,Africa,35,27,e6955a2c59dc9083,Asia,129,...,3,2,b996521be9c996db,Russia,102,75,eb51f8315cf769b3,South America,31,20
1,2022-01-02 00:00:00+00:00,355,355,c4a371acd7c3df8b,Africa,40,31,e6955a2c59dc9083,Asia,127,...,3,2,b996521be9c996db,Russia,108,75,eb51f8315cf769b3,South America,32,20
2,2022-01-03 00:00:00+00:00,348,348,c4a371acd7c3df8b,Africa,45,35,e6955a2c59dc9083,Asia,123,...,3,2,b996521be9c996db,Russia,97,69,eb51f8315cf769b3,South America,33,20
3,2022-01-04 00:00:00+00:00,353,353,c4a371acd7c3df8b,Africa,44,35,e6955a2c59dc9083,Asia,119,...,3,2,b996521be9c996db,Russia,107,72,eb51f8315cf769b3,South America,35,20
4,2022-01-05 00:00:00+00:00,364,364,c4a371acd7c3df8b,Africa,42,33,e6955a2c59dc9083,Asia,110,...,3,2,b996521be9c996db,Russia,104,77,eb51f8315cf769b3,South America,42,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,2022-12-27 00:00:00+00:00,404,404,c4a371acd7c3df8b,Africa,75,46,e6955a2c59dc9083,Asia,198,...,2,2,b996521be9c996db,Russia,100,83,eb51f8315cf769b3,South America,52,30
361,2022-12-28 00:00:00+00:00,401,401,c4a371acd7c3df8b,Africa,80,49,e6955a2c59dc9083,Asia,180,...,2,2,b996521be9c996db,Russia,99,84,eb51f8315cf769b3,South America,43,25
362,2022-12-29 00:00:00+00:00,407,407,c4a371acd7c3df8b,Africa,86,52,e6955a2c59dc9083,Asia,178,...,1,1,b996521be9c996db,Russia,94,80,eb51f8315cf769b3,South America,42,25
363,2022-12-30 00:00:00+00:00,413,413,c4a371acd7c3df8b,Africa,85,53,e6955a2c59dc9083,Asia,202,...,,,b996521be9c996db,Russia,102,84,eb51f8315cf769b3,South America,38,22


In [20]:
transform_df2 = transform_df(df2,'origin_region')

In [21]:
transform_df2[transform_df2['date']=='2022-01-03'][['count','value']].sum()

count    561
value    341
dtype: object

In [22]:
px.bar(transform_df2,x = 'date', y = 'count',color = 'origin_region',title = 'Laden sailing Aframax split by origin_region')

## Example: Laden sailing vessels carrying Urals by ves class

In [23]:
df_temp = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    breakdown_split_property = 'vessel_class_granular',
    time_min = datetime(2022,1,1),
    time_max = datetime.today(),
    movement_status = 'moving',
    latest_products = urals,
    #locations = egypt,
    voyage_status = 'laden'
    ).to_df(columns = 'all')

In [24]:
df_temp

,key,value,count,breakdown.0.id,breakdown.0.label,breakdown.0.count,breakdown.0.value,breakdown.1.id,breakdown.1.label,breakdown.1.count,...,breakdown.4.count,breakdown.4.value,breakdown.5.label,breakdown.5.id,breakdown.5.value,breakdown.5.count,breakdown.6.label,breakdown.6.id,breakdown.6.value,breakdown.6.count
0,2022-01-01 00:00:00+00:00,22,22,oil_aframax,oil_aframax,13,13,oil_lr2,oil_lr2,8,...,1,1,oil_vlcc,oil_vlcc,,,,,,
1,2022-01-02 00:00:00+00:00,22,22,oil_aframax,oil_aframax,11,11,oil_lr2,oil_lr2,8,...,3,3,oil_vlcc,oil_vlcc,,,,,,
2,2022-01-03 00:00:00+00:00,22,22,oil_aframax,oil_aframax,13,13,oil_lr2,oil_lr2,7,...,2,2,oil_vlcc,oil_vlcc,,,,,,
3,2022-01-04 00:00:00+00:00,22,22,oil_aframax,oil_aframax,12,12,oil_lr2,oil_lr2,8,...,2,2,oil_vlcc,oil_vlcc,,,,,,
4,2022-01-05 00:00:00+00:00,23,23,oil_aframax,oil_aframax,13,13,oil_lr2,oil_lr2,8,...,2,2,oil_vlcc,oil_vlcc,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,2024-06-20 00:00:00+00:00,58,58,oil_aframax,oil_aframax,31,31,oil_lr2,oil_lr2,15,...,10,10,oil_vlcc,oil_vlcc,1,1,,,,
902,2024-06-21 00:00:00+00:00,66,66,oil_aframax,oil_aframax,36,36,oil_lr2,oil_lr2,16,...,12,12,oil_vlcc,oil_vlcc,1,1,,,,
903,2024-06-22 00:00:00+00:00,62,62,oil_aframax,oil_aframax,32,32,oil_lr2,oil_lr2,15,...,12,12,oil_vlcc,oil_vlcc,1,1,,,,
904,2024-06-23 00:00:00+00:00,62,62,oil_aframax,oil_aframax,33,33,oil_lr2,oil_lr2,14,...,12,12,oil_vlcc,oil_vlcc,1,1,,,,


In [25]:
transform_df_temp = transform_df(df_temp,'v_class')

In [26]:
px.bar(transform_df_temp,x = 'date', y = 'count',color = 'v_class',title = 'Laden sailing Urals at Egypt vessels split by v_class')

## Customised status for stationary vessel (loading/discharging, floating storage, shipyard)

## 3. Stationary vessel waiting to be discharged at berth

In [27]:
df3 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    time_min = datetime(2022,1,1),
    #vessels = 'aframax',
    time_max = datetime(2022,12,31,23,59,59),
    movement_status = 'waiting',
    cargo_status = 'discharging',
    location_status = 'berth', ## anchorage-zone, dry-dock
    voyage_status = 'laden'
    ).to_df(columns = 'all')

In [28]:
px.bar(df3, x= 'key', y= 'count',title = 'Number of stationary vessel waiting to be discharged at berth')

## 4. Stationary vessel acting as floating storage

In [29]:
df4 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    time_min = datetime(2022,1,1),
    vessels = 'oil_aframax_lr2',
    time_max = datetime(2023,9,30),
    movement_status = 'stationary',
    cargo_status = 'floating-storage',
    voyage_status = 'laden'
    ).to_df(columns = 'all')

In [30]:
px.line(df4, x = 'key', y = 'count', title = 'number of stationary vessel acting as floating storage')

## 5. Stationary vessel waiting at shipyard

In [31]:
df5 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    time_min = datetime(2022,1,1),
    vessels = 'oil_aframax_lr2',
    time_max = datetime(2023,9,30),
    movement_status = 'stationary',
    location_status = 'dry-dock',
    voyage_status = 'ballast'
    ).to_df(columns = 'all')

In [32]:
px.line(df5, x = 'key', y = 'count', title = 'number of stationary vessel waiting at shipyard')

## Congestion

## 6. Number of ships congested at Mumbai port

In [33]:
# Extracting mumbai port
mumbai = v.Geographies().search('Mumbai [IN]',True).to_df().loc[:,'id'].to_list()
india = v.Geographies().search('India',True).to_df().loc[:,'id'].to_list()
assert len(mumbai)==1

In [34]:
df6 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'vessel_count',
    breakdown_unit_operator = 'sum',
    time_min = datetime(2022,1,1),
    time_max = datetime(2023,3,1,23,59,59),
    congestion_target_location= mumbai,
    movement_status = ['congestion']
    ).to_df(columns = 'all')

In [35]:
px.line(df6, x = 'key', y = 'count', title = 'number of vessels waiting at mumbai port')

In [36]:
df8 = v.VoyagesTimeseries().search(
    breakdown_frequency = 'day',
    breakdown_property = 'avg_wait_time',
    breakdown_unit_operator = 'avg',
    time_min = datetime(2022,1,1),
    time_max = datetime(2023,12,31,23,59,59),
    congestion_target_location = mumbai,
    exclude_overlapping_entries = False,
    movement_status = ['congestion']
    ).to_df(columns = 'all')

In [37]:
# Convert from seconds to days
df8['value'] = df8['value'].replace({"":0}).astype(int)
df8['value'] = df8['value']/(24*3600)
px.line(df8,x= 'key', y = 'value',title = 'Mumbai average waiting time (days)')

## Freight Pricing

In [38]:
df7 = v.FreightPricingTimeseries().search(
    time_min=datetime(2023,1,1),
    time_max=datetime.today(),
    routes=['TD3C'],
    breakdown_property='rate',
    breakdown_frequency='day').to_df()

In [39]:
df7

,key,value,count
0,2023-01-01 00:00:00+00:00,,
1,2023-01-02 00:00:00+00:00,,
2,2023-01-03 00:00:00+00:00,56.41,1.0
3,2023-01-04 00:00:00+00:00,55.16,1.0
4,2023-01-05 00:00:00+00:00,53.27,1.0
...,...,...,...
536,2024-06-20 00:00:00+00:00,49.900002,1.0
537,2024-06-21 00:00:00+00:00,49.549999,1.0
538,2024-06-22 00:00:00+00:00,,
539,2024-06-23 00:00:00+00:00,,


In [40]:
px.line(df7, x = 'key', y = 'value')

## Summary

The notebook above shows:
1. Preview of voyage data (VoyageSearchEnriched & VoyageTimeSeries)
2. Freight metrics split by voyage status, products
3. Other status for stationary vessels (berth, anchorage-zone, dry-dock)
4. Congestion
5. Freight Pricing